In [53]:
# imports
import matplotlib
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.ndimage import zoom
import json
import warnings
from random import randint
import random
import SimpleITK as sitk
from multi_slice_viewer import multi_slice_viewer
import tensorflow as tf
from keras.models import Model, load_model
from keras.layers import Input, Conv3D, MaxPooling3D, Dropout, Conv3DTranspose, UpSampling3D, concatenate, Cropping3D, Reshape, BatchNormalization
import keras.callbacks
from keras import backend as K
from keras import optimizers
from keras import regularizers
from keras.optimizers import SGD, Adam
from keras.utils.np_utils import to_categorical
from IPython.display import clear_output
import gzip 
from tensorflow.python.client import device_lib

#this part is needed if you run the notebook on Cartesius with multiple cores
n_cores = 32
config = tf.ConfigProto(intra_op_parallelism_threads=n_cores-1, inter_op_parallelism_threads=1, allow_soft_placement=True)
session = tf.Session(config=config)
K.set_session(session)
os.environ["OMP_NUM_THREADS"] = str(n_cores-1)
os.environ["KMP_BLOCKTIME"] = "1"
os.environ["KMP_SETTINGS"] = "1"
os.environ["KMP_AFFINITY"]= "granularity=fine,verbose,compact,1,0"

In [2]:
# Task03_liver dir in same directory as notebook
data_path = './Task03_Liver/'

In [3]:
# info about dataset in json file
with open(data_path + 'dataset.json') as f:
    d = json.load(f)   
    
    # paths to training set images with label
    train_paths = d['training']
    
    # paths to testset images with label
    test_paths = d['test'] 

In [4]:
# change to data dir 
os.chdir(data_path)
print(os.getcwd())

/nfs/home3/hesterh/ISMI_project/Task03_Liver


(131,)


In [48]:
# load images and labels, loading all takes some time
train_imgs = [sitk.ReadImage(train_instance['image']) for train_instance in train_paths[0:10]]
train_lbls = [sitk.ReadImage(train_instance['label']) for train_instance in train_paths[0:10]]

In [49]:
# train images as numpy
np_train_imgs = [sitk.GetArrayFromImage(i) for i in train_imgs]
np_train_lbls = [sitk.GetArrayFromImage(i) for i in train_lbls]

In [28]:
def resample(np_imgs, spacings):
    """
    Resample to 1mm x 1mm x 1mm. 
    np_imgs: list of images or labels to be resampled as numpy
    spacings: spacings to resample with, order: (z, x, y)
    """ 
    resampled = []
    
    for i in range(len(np_imgs)): 
        # apply zoom with spacing, no clue what spline interpolation is
        resampled.append(zoom(np_imgs[i], spacings[i], order=1))

    return resampled

In [50]:
# store shapes before to check
shapes_before = [img.shape for img in np_train_imgs]

# spacings from sitk images
spacings = [img.GetSpacing() for img in train_imgs]

# change order
spacings = [(z, x, y) for x, y, z in spacings]          # order: (z, x, y)

In [51]:
# resample train images and labels
np_train_imgs = resample(np_train_imgs, spacings)
np_train_lbls = resample(np_train_lbls, spacings)

/hpc/sw/python-3.5.2/lib/python3.5/site-packages/scipy/ndimage/interpolation.py:616: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


In [52]:
# lets print what happened.
print("Before\t\t\tSpacings\t\tAfter")
for i in range(len(np_train_imgs)):    
    # round for printing
    spacing_round = [(round(a, 1), round(b, 1), round(c, 1)) for a, b, c in spacings]    
    print("{}\t\t{}\t\t{}".format(shapes_before[i] , spacing_round[i], np_train_imgs[i].shape))

Before			Spacings		After
(588, 512, 512)		(1.0, 0.7, 0.7)		(588, 351, 351)
(245, 512, 512)		(2.0, 0.8, 0.8)		(490, 386, 386)
(94, 512, 512)		(5.0, 0.8, 0.8)		(470, 400, 400)
(424, 512, 512)		(1.5, 0.7, 0.7)		(636, 381, 381)
(845, 512, 512)		(0.8, 0.9, 0.9)		(676, 472, 472)
(513, 512, 512)		(0.8, 0.6, 0.6)		(410, 320, 320)
(200, 512, 512)		(1.0, 1.0, 1.0)		(200, 512, 512)
(908, 512, 512)		(0.7, 0.8, 0.8)		(636, 404, 404)
(856, 512, 512)		(0.8, 0.7, 0.7)		(685, 373, 373)
(105, 512, 512)		(4.0, 0.9, 0.9)		(420, 435, 435)


In [ ]:
with gzip.open('liver_data.gz', 'ab') as f:
    f.write(

In [41]:
with open('../liver.pickle', 'a+b') as handle:
    pickle.dump({'images': np_train_imgs, 'labels': np_train_lbls}, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [42]:
with open('../liver.pickle', 'rb') as handle:
    data = pickle.load(handle)
images = data['images']
labels = data['labels']

In [43]:
print(np.shape(images))

(10,)
